## Advent of code 2019 day 21-25
See https://adventofcode.com/

In [ ]:
# note that this notebook requires the .venv-pypy environment for pypy 3.9
# to activate it from a git bash shell: source .venv-pypy/Scripts/activate
# to generate its requirements: pip freeze > .venv-pypy-requirements.txt

import collections
import itertools
import re
import copy
import math
import sys
import time
import json
import heapq
import cProfile

In [ ]:
# utility functions and version check

def get_line_groups(lines):
    '''return list of lists of lines, each separated by empty lines, ignores empty lines from start and end'''
    lines=list(lines)
    lines.append('') # add terminator
    res=[]
    group=[]
    for line in lines:
        line=line.strip()
        if len(line)>0:
            group.append(line)
        elif len(group)>0: # close group
            res.append(group)
            group=[]
    return res

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

def exit():
    raise StopExecution()
    
print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}') # supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # 2019 day 2), then whenever you want save
# a private leaderboard json file, and run python privaterank.py filename.json

In [ ]:
# 2019 day 21
# start_ts=1668846011
# mv ~/Downloads/input* data_src/2019-day-21-input.txt
# big input file looks like: IntCode
# idea: part 1 parse ..., then ...

class Computer:
    def __init__(self):
        self.relbase=0
        self.i_ptr=0
        self.OFFSET_DIVS={1: 100, 2: 1000, 3: 10000}
        self.board=None

    def fetch_param(self, data, opcode, i, offset):
        mode=(opcode//self.OFFSET_DIVS[offset]) % 10
        param=data[i+offset]
        if mode==0: # position mode
            return data[param]
        elif mode==1: # immediate mode
            return param
        elif mode==2: # relative mode
            return data[param+self.relbase]
        else:
            print(f'invalid mode {mode}')
            assert False

    def store_param(self, data, opcode, i, offset, newval):
        mode=(opcode//self.OFFSET_DIVS[offset]) % 10
        param=data[i+offset]
        if mode==0: # position mode
            data[param]=newval
        elif mode==2: # relative mode
            data[param+self.relbase]=newval
        else:
            print(f'invalid mode {mode}')
            assert False

    def run_opcodes(self, data): # HALTS ON INPUT -1 !
        i=self.i_ptr
        halted=False
        while True:
            if data[i]%100==1: # add
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, a+b)
                i+=4
            elif data[i]%100==2: # mult
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, a*b)
                i+=4
            elif data[i]%100==3: # input
                a=self.get_input()
                if a==-1:
                    halted=True
                    break
                self.store_param(data, data[i], i, 1, a)
                i+=2
            elif data[i]%100==4: # output
                a=self.fetch_param(data, data[i], i, 1)
                self.put_output(a)
                i+=2
            elif data[i]%100==5: # jump-if-true
                a=self.fetch_param(data, data[i], i, 1)
                if a!=0:
                    i=self.fetch_param(data, data[i], i, 2)
                else:
                    i+=3
            elif data[i]%100==6: # jump-if-false
                a=self.fetch_param(data, data[i], i, 1)
                if a==0:
                    i=self.fetch_param(data, data[i], i, 2)
                else:
                    i+=3
            elif data[i]%100==7: # less-than
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, 1 if a<b else 0)
                i+=4
            elif data[i]%100==8: # equals
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, 1 if a==b else 0)
                i+=4
            elif data[i]%100==9: # relbase
                a=self.fetch_param(data, data[i], i, 1)
                self.relbase+=a
                i+=2
            elif data[i]%100==99: # stop
                halted=True
                break
            else:
                print(f'unknown instruction {data[i]} at position {i}')
                assert False
        self.i_ptr=i
        return

    def init_board(self, do_part):
        assert self.board is None
        self.board={} # maps (x,y) cell to ascii code
        self.do_part=do_part
        self.px=0
        self.py=0
        self.part1plan=None # used for both parts
        self.part1planpos=0
        self.part1res=None

    def get_input(self):
        if self.board is None:
            self.init_board()
        #if self.do_part==1 or self.do_part==2:
        c=self.part1plan[self.part1planpos]
        self.part1planpos+=1
        return ord(c)

    def put_output(self, a):
        if self.board is None:
            self.init_board()
        # if self.do_part==1: # same for both parts
        if a==10:
            self.px=0
            self.py+=1
        elif a<=255:
            self.board[ (self.px, self.py) ]=a
            self.px+=1
        else:
            self.part1res=a

    def print_painted(self, noprint=False):
        y_vals=[tup[1] for tup in self.board.keys()]
        x_vals=[tup[0] for tup in self.board.keys()]
        res=[]
        for y in range(min(y_vals), max(y_vals)+1):
            row=''
            for x in range(min(x_vals), max(x_vals)+1):
                c=self.board.get( (x, y) , -1)
                row+=chr(c) if c>=32 else '?'
            if not noprint:
                print(row)
            res.append(row)
        return res

# jumps if ground at 4 and not at 1 or 2 or 3
p1cmd1='''
OR A T
AND B T
AND C T
NOT T J
AND D J
WALK
'''

# jumps if ground at 4 and not at (1 and 2 and 3)
# except if after jump ground not at 1 and not at 4, ie except if ground not at 5 and not at 8,
# ie and there must be ground at 5 or 8
p2cmd1='''
OR A T
AND B T
AND C T
NOT T J
AND D J
AND E T
OR E T
OR H T
AND T J
RUN
'''

sample1=open('data_src/2019-day-21-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data0=[ int(s) for s in lines[0].split(',') ] # program
for do_part in [1,2]:
    print(f'{do_part=}')
    data=collections.defaultdict(lambda : 0)
    for i,v in enumerate(data0): # converted to large mem
        data[i]=v
    cmp=Computer()
    cmp.init_board(do_part)
    if do_part==1:
        cmp.part1plan=p1cmd1.replace('\n', '', 1)
    else:
        cmp.part1plan=p2cmd1.replace('\n', '', 1)
    cmp.run_opcodes(data)
    if do_part==1 or do_part==2:
        sample2_strs=cmp.print_painted()
        part1res=cmp.part1res
        print(f'part {do_part} {part1res=}')

# part 1: 19360288
# part 2: 1143814750


In [ ]:
# TEMPLATE
# 2019 day 6
# start_ts=RUN FIRST CELL TO GET TIME CODE BEFORE OPENING THE ASSIGNMENT
# mv ~/Downloads/input* data_src/2019-day-6-input.txt
# big input file looks like: 
# idea: part 1 parse ..., then ...

sample1='''

'''

#sample1=open('data_src/2019-day-6-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ int(s) for s in lines[0].split(',') ]
groups=get_line_groups(lines)
data0=[ s.split() for s in lines ]
data0=[ [cmd, int(num), 0] for cmd, num in data0 ]
data=[ result.group(1, 2, 3, 4, 5, 6, 7) for s in lines if (result:= re.match(r'(\w+)\s*x=([\d\-]+)\.\.([\d\-]+),y=([\d\-]+)\.\.([\d\-]+),z=([\d\-]+)\.\.([\d\-]+)', s)) ]
data=[ (row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6]) ) for row in data ]
# template, remove what's not needed